In [1]:
import pandas as pd
import numpy as np
from threading import Thread
import spacy
import csv
nlp = spacy.load('en_core_web_lg')

2021-08-01 17:22:24.431563: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-01 17:22:24.431583: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
file_path = 'train_800k.csv'
train_data = pd.read_csv(file_path, index_col=0)

In [3]:
file_path = './dataset/test.csv'
escape_char_defined = "\\"
test_data = pd.read_csv(file_path, error_bad_lines=False, quoting=csv.QUOTE_NONE, escapechar=escape_char_defined)
test_data.head()

/home/abhishek/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [4]:
test_data = test_data.replace(np.nan, '', regex=True)

In [5]:
train_data = train_data.dropna(axis=0)

In [6]:
# train_data = train_data.head(8000)

In [7]:
# test_data = test_data.head(8000)

In [8]:
test_data.shape

(8000, 5)

In [9]:
train_data.shape

(8000, 5)

In [10]:
threads = []

## train data test

In [11]:
def get_title_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            filtered_text = ""
            doc = nlp(data["TITLE"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global title_vector
    title_vector = np.array(lst)

# threads.append(Thread(target=get_title_vec))

In [12]:
def get_desc_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            filtered_text = ""
            doc = nlp(data["DESCRIPTION"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global desc_vector
    desc_vector = np.array(lst)
    
# threads.append(Thread(target=get_desc_vec))

In [13]:
def get_brand_vec():
    lst = []
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            doc = nlp(data["BRAND"])
            lst.append(doc.vector)
    global brand_vector
    brand_vector = np.array(lst)
    
# threads.append(Thread(target=get_brand_vec))

In [14]:
def get_bp_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            filtered_text = ""
            doc = nlp(data["BULLET_POINTS"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global bp_vector
    bp_vector = np.array(lst)
    
threads.append(Thread(target=get_bp_vec))

## test data threads

In [15]:
def get_title_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in test_data.iterrows():
            filtered_text = ""
            doc = nlp(data["TITLE"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global t_title_vector
    t_title_vector = np.array(lst)

# threads.append(Thread(target=get_title_vec))

In [16]:
def get_desc_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            filtered_text = ""
            doc = nlp(data["DESCRIPTION"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global t_desc_vector
    t_desc_vector = np.array(lst)
    
# threads.append(Thread(target=get_desc_vec))

In [17]:
def get_brand_vec():
    lst = []
    with nlp.disable_pipes():
        for idx, data in train_data.iterrows():
            doc = nlp(data["BRAND"])
            lst.append(doc.vector)
    global t_brand_vector
    t_brand_vector = np.array(lst)
    
# threads.append(Thread(target=get_brand_vec))

In [18]:
def get_bp_vec():
    lst=[]
    with nlp.disable_pipes():
        for idx, data in test_data.iterrows():
            filtered_text = ""
            doc = nlp(data["BULLET_POINTS"])
            for token in doc:
                if(token.is_stop==False):
                    filtered_text = filtered_text+" "+str(token)
            lst.append(nlp(filtered_text).vector)
    global t_bp_vector
    t_bp_vector = np.array(lst)
    
threads.append(Thread(target=get_bp_vec))

In [19]:
threads

[<Thread(Thread-7, initial)>, <Thread(Thread-8, initial)>]

In [20]:
def run_threads():
    for t in threads:
        t.start()
    for t in threads:
        t.join()
#     A = (title_vector, desc_vector, brand_vector, bp_vector)
    global X
#     X = np.hstack(A)
    X = bp_vector
#     B = (t_title_vector, t_desc_vector, t_brand_vector, t_bp_vector)
    global t_X
#     t_X = np.hstack(A)
    t_X = t_bp_vector
    

In [21]:
run_threads()
X.shape

(8000, 300)

In [22]:
from sklearn.svm import LinearSVC
model = LinearSVC(random_state=1, dual=False)
model.fit(X, train_data["BROWSE_NODE_ID"])

LinearSVC(dual=False, random_state=1)

In [23]:
preds = model.predict(t_X)
preds

array([  74, 1045, 2821, ..., 1045,  507,  709])

In [24]:
df = pd.DataFrame()
df["PRODUCT_ID"] = test_data["PRODUCT_ID"]
df["BROWSE_NODE_ID"] = preds
df

,PRODUCT_ID,BROWSE_NODE_ID
0,1,74
1,2,1045
2,3,2821
3,4,1045
4,5,1203
...,...,...
7995,7996,977
7996,7997,1045
7997,7998,1045
7998,7999,507


In [25]:
df.to_csv('submit_800k-1.csv',index=False)
